In [ ]:
import os

directory = "headers"


if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
%%writefile ./headers/config_t.h

/*Struct with the configuration passed by the file*/
#ifndef CONFIG_H
	#define CONFIG_H
	typedef struct config_t {
		unsigned short int GEN_PROC_CONEJOS;
		unsigned short int GEN_PROC_ZORROS;
		unsigned short int GEN_COMIDA_ZORROS;
		unsigned int N_GEN;
		unsigned int L;
		unsigned int C;
		unsigned int GEN;
	} config_t;
#endif

Overwriting ./headers/config_t.h


In [ ]:
%%writefile ./headers/coord_t.h

/*A simple coordinate struct*/

#ifndef COORD_H
	#define COORD_H
	typedef struct coord_t {
		int x;
		int y;
	} coord_t;
#endif

Overwriting ./headers/coord_t.h


In [ ]:
%%writefile ./headers/fileHandler.h

/*Functions to handle interactions with files*/

#ifndef FILEHANDLER_H
	#define FILEHANDLER_H

	#include "config_t.h"
	#include "coord_t.h"
	#include "types_t.h"
	#include <stdlib.h>
	#include <stdio.h>
	#include <errno.h>
	#include <string.h>

	int readFile (char *filename, config_t *c, object_t ***eco);
#endif

Overwriting ./headers/fileHandler.h


In [ ]:
%%writefile ./headers/rules.h

#ifndef RULES_H
	#define RULES_H
	#include "types_t.h"
	#include "coord_t.h"
	#include "config_t.h"
	#include <omp.h>

    #define MIN(a, b) ((a) < (b)) ? (a) : (b)
    #define MAX(a, b) ((a) > (b)) ? (a) : (b)

	void rabbit_rules (object_t **eco, config_t conf, coord_t p);
	void fox_rules (object_t **eco, config_t conf, coord_t p);
	void conflict (object_t **eco, config_t conf, int type);

	animal_t choose_rabbit(object_t object);
	animal_t choose_fox(object_t object);

#endif

Overwriting ./headers/rules.h


In [ ]:
%%writefile ./headers/types_t.h

#ifndef TYPES_H
	#define TYPES_H
	
	#include <stdlib.h>
	#include <stdio.h>
	#include <errno.h>
	#include <string.h>
	#include <omp.h>
	#include "coord_t.h"
	#include "config_t.h"
	
	#define EMPTY 0
	#define RABBIT 1
	#define FOX 2
	#define ROCK 3

	#define UP 0
	#define RIGHT 1
	#define DOWN 2
	#define LEFT 3
	#define CENTER 4

	/* Macros for deferencing pointers*/
	#define RABBIT_P(A) ((rabbit_t*)A)
	#define FOX_P(A) ((fox_t*)A)
	
	#define _RED     "\033[1m\033[31m"
	#define _GREEN   "\033[1m\033[32m"
	#define _YELLOW  "\033[1m\033[33m"
	#define _BLUE    "\033[1m\033[34m"
	#define _MAGENTA "\033[1m\033[35m"
	#define _CYAN    "\033[1m\033[36m"
	#define _RESET   "\x1b[0m"

	
	#define NEWANIMAL(TYPE, GEN_NASCIMENTO, GEN_COMIDA) (animal_t){ \
					  .type = TYPE, \
					  .gen_nascimento = GEN_NASCIMENTO, \
					  .gen_comida = GEN_COMIDA}

	typedef struct animal_t {
		unsigned short int type;
		unsigned int gen_nascimento;
		unsigned int gen_comida;
	} animal_t;

	typedef struct object_t {
		animal_t animal;
		animal_t *candidates;
	} object_t;

	object_t** allocateMatrix (int X, int Y);
	void printMatrix (object_t **eco, int X, int Y, config_t conf);
	int insert_into_matrix (object_t **matrix, char* object_type, coord_t p);
	void deepcopy (object_t** A, object_t** B, int L, int C);

	/*These functions transforms the object into a new_'entity'*/
	void clear_position (object_t *object);

#endif

Overwriting ./headers/types_t.h


In [ ]:
%%writefile fileHandler.c

#include "headers/fileHandler.h"


/*Reads the file passed by the string filename returning a configuration struct and the ecosystem matrix*/
int readFile (char *filename, config_t *conf, object_t ***eco) {
	FILE *fp;
	unsigned int N;
	char object_type[7];
	coord_t p;
	object_t  **matrix;

	fp = fopen(filename, "r");
	if (fp == NULL) {
		perror(filename);
		exit(1);
	}

	/*TODO -- file checking*/
	fscanf(fp, "%hu %hu %hu %u %u %u %u\n",&(conf->GEN_PROC_CONEJOS),
										&(conf->GEN_PROC_ZORROS),
										&(conf->GEN_COMIDA_ZORROS),
										&(conf->N_GEN),
										&(conf->L),
										&(conf->C),
										&N);
	conf->GEN = 0;
	
	matrix = allocateMatrix(conf->L, conf->C);

	for (int i = 0; i < N; i++) {
		fscanf(fp, "%s %d %d\n", object_type, &(p.x), &(p.y));
		insert_into_matrix(matrix, object_type, p);
	}

	*eco = matrix;
	
	return 1;
}

Overwriting fileHandler.c


In [ ]:
%%writefile rules.c

#include "headers/rules.h"
#include <time.h>

//find an slot of type 'type'
coord_t neighbour(object_t **eco, config_t conf, coord_t p, int type) {
	srand(time(NULL));
	coord_t possible[4];
	int possible_n = 0;

	//left
	if (p.y - 1 >= 0 && eco[p.x][p.y - 1].animal.type == type)
		possible[possible_n++] = (coord_t){.x = p.x, .y = p.y - 1};

	//down
	if (p.x + 1 < conf.L && eco[p.x + 1][p.y].animal.type == type)
		possible[possible_n++] = (coord_t){.x = p.x + 1, .y = p.y};

	//right
	if (p.y + 1 < conf.C && eco[p.x][p.y + 1].animal.type == type)
		possible[possible_n++] = (coord_t){.x = p.x, .y = p.y + 1};

	//up
	if (p.x - 1 >= 0 && eco[p.x - 1][p.y].animal.type == type)
		possible[possible_n++] = (coord_t){.x = p.x - 1, .y = p.y};

	if (possible_n > 0){
		if (possible_n > 1)
			return possible[(conf.GEN + p.x +p.y) % possible_n];		//choose one of the neighbours
		else
			return possible[0];
	}
	else 
		return (coord_t){.x = -1, .y = -1};
}

/*******************************
--------- Rabbit Rules ---------
********************************/

// Inserts a rabbit and resolve possible conflicts
void rabbit_insert (object_t **eco, animal_t r, coord_t old, coord_t new) {
	int candidate_id;

	if (old.y + 1 == new.y)
		candidate_id = LEFT;
	else if (old.x - 1 == new.x)
		candidate_id = DOWN;
	else if (old.y - 1 == new.y)
		candidate_id = RIGHT;
	else if (old.x + 1 == new.x)
		candidate_id = UP;
	else if (old.x == new.x && old.y == new.y)
		candidate_id = CENTER;

	eco[new.x][new.y].candidates[candidate_id] = r;
}

// Make a move if a available slot is found and returns the new position
int rabbit_move (object_t **eco, config_t conf, coord_t p) {
	coord_t new = neighbour(eco, conf, p, EMPTY);	

	if (new.x >= 0 && new.y >= 0){
		if (conf.GEN - eco[p.x][p.y].animal.gen_nascimento > conf.GEN_PROC_CONEJOS) {		// Reproduction 
			rabbit_insert(eco, NEWANIMAL(RABBIT, conf.GEN, -1), p, p);			
			rabbit_insert(eco, NEWANIMAL(RABBIT, conf.GEN, -1), p, new);
		} 
		else {																// Move without reproducing 
			//rabbit_insert(eco, eco[p.x][p.y].animal, p, new);
			rabbit_insert(eco, NEWANIMAL(RABBIT, eco[p.x][p.y].animal.gen_nascimento, -1), p, new);
		}
		return 1;
	} 										//No neighbours, returns false
	return 0;
}

void rabbit_rules (object_t **eco, config_t conf, coord_t p) {
	// Movement 
	rabbit_move (eco, conf, p);
	return;
}


/*******************************
---------- Fox Rules -----------
********************************/


/*Insert the fox f into the next_eco in the position p, resolving conflicts if they ocour*/
void fox_insert (object_t **eco, animal_t f, coord_t old, coord_t new) {
	int candidate_id;

	if (old.y + 1 == new.y)
		candidate_id = UP;
	else if (old.x - 1 == new.x)
		candidate_id = RIGHT;
	else if (old.y - 1 == new.y)
		candidate_id = DOWN;
	else if (old.x + 1 == new.x)
		candidate_id = LEFT;
	else if (old.x == new.x && old.y == new.y)
		candidate_id = CENTER;

	eco[new.x][new.y].candidates[candidate_id] = f;
}

int predation(object_t **eco, config_t conf, coord_t p) {
	coord_t new = neighbour(eco, conf, p, RABBIT);

	if (new.x >= 0 && new.y >= 0){
		if (conf.GEN - eco[p.x][p.y].animal.gen_nascimento > conf.GEN_PROC_ZORROS) {		/* Reproduction */
			fox_insert(eco, NEWANIMAL(FOX, conf.GEN, conf.GEN), p, p);
			fox_insert(eco, NEWANIMAL(FOX, conf.GEN, conf.GEN), p, new);
		} 
		else {																/* Move without reproducing */
			fox_insert(eco, NEWANIMAL(FOX, eco[p.x][p.y].animal.gen_nascimento, conf.GEN), p, new);
		}
		return 1;
	}
	return 0;
}

/*TODO: fox predation, fox move, fox death*/

int fox_move (object_t **eco, config_t conf, coord_t p) {
	coord_t new = neighbour(eco, conf, p, EMPTY);

	if (new.x >= 0 && new.y >= 0){
		if (conf.GEN - eco[p.x][p.y].animal.gen_nascimento > conf.GEN_PROC_ZORROS) {		/* Reproduction */
			fox_insert(eco, NEWANIMAL(FOX, conf.GEN, conf.GEN), p, p);
			fox_insert(eco, NEWANIMAL(FOX, conf.GEN, conf.GEN), p, new);
		} else {																/* Move without reproducing */
			fox_insert(eco, eco[p.x][p.y].animal, p, new);
		}
		return 1;
	}
	return 0;
}

void fox_rules (object_t **eco, config_t conf, coord_t p) {
	/* Movement */ 
	if (!predation(eco, conf, p)) {
		if(conf.GEN - eco[p.x][p.y].animal.gen_comida >= conf.GEN_COMIDA_ZORROS){		//death
			//clear_position(&(next_eco[p.x][p.y]));
		}
		else
			fox_move(eco, conf, p);
	}
}

animal_t choose_rabbit (object_t object) {
	animal_t old_animal = object.animal;

	animal_t *animals = object.candidates;
	animal_t animal;

	int success = 0;
	for (int i = 0; i < 5; i++){
		if (animals[i].type != RABBIT)
			continue;

		success += 1;
		if (success == 1)
			animal = animals[i];

		if (animal.gen_nascimento < animals[i].gen_nascimento)
			continue;
		else
			animal = animals[i];
		animals[i] = NEWANIMAL(EMPTY, -1, -1);

	}
	if (success)
		return animal;
	else if (object.animal.type != RABBIT)
		return old_animal;
	else 
		return NEWANIMAL(EMPTY, -1, -1);
}

animal_t choose_fox (object_t object) {
	animal_t old_animal = object.animal;

	animal_t *animals = object.candidates;
	animal_t animal;

	int success = 0;
	for (int i = 0; i < 5; i++){
		if (animals[i].type != FOX)
			continue;
		
		success += 1;
		if (success == 1)
			animal = animals[i];

		if (animal.gen_nascimento < animals[i].gen_nascimento){
			continue;
		}
		else if (animal.gen_nascimento > animals[i].gen_nascimento){
			animal = animals[i];
		} 
		else {
			if (animal.gen_comida > animals[i].gen_comida){
				continue;
			}
			else {
				animal = animals[i];
			}
		}
		animals[i] = NEWANIMAL(EMPTY, -1, -1);
	}
	if (success)
		return animal;
	else if (object.animal.type != FOX)
		return old_animal;
	else 
		return NEWANIMAL(EMPTY, -1, -1);
}

Overwriting rules.c


In [ ]:
%%writefile types.c

#include "headers/types_t.h"

object_t **allocateMatrix (int L, int C) {
	object_t **matrix = (object_t**) malloc(L*sizeof(object_t*));
	for (int i = 0; i < L; i++) {
		matrix[i] = (object_t*) calloc(C,sizeof(object_t));
		for (int j = 0; j < C; j++) {
			matrix[i][j].candidates = malloc(5*sizeof(animal_t));
		}
	}
	return matrix;
}

void printMatrix (object_t **eco, int X, int Y, config_t conf) {
	for (int i = 0; i < X; i++){
		printf("|");
		for (int j = 0; j < Y; j++) {
			if (eco[i][j].animal.type == EMPTY)
				printf("   ");
			else if (eco[i][j].animal.type == ROCK)
				printf(" * ");
			else if (eco[i][j].animal.type == RABBIT)
				printf(_GREEN"R%-2d"_RESET, conf.GEN - eco[i][j].animal.gen_nascimento);
			else if (eco[i][j].animal.type == FOX)
				printf(_RED"F%d%d"_RESET, conf.GEN - eco[i][j].animal.gen_nascimento, conf.GEN - eco[i][j].animal.gen_comida);
			else
				printf("?");
			printf("|");
		}
		printf("\n");
	}
	printf("---------------------\n");
}

int insert_into_matrix (object_t **matrix, char* object_type, coord_t p) {
	if (strcmp(object_type, "CONEJO") == 0) {
		matrix[p.x][p.y].animal = NEWANIMAL(RABBIT, 0, -1);
	}
	else if (strcmp(object_type, "ZORRO") == 0) {
		matrix[p.x][p.y].animal = NEWANIMAL(FOX, 0, 0);
	}
	else if (strcmp(object_type, "ROCA") == 0) {
		matrix[p.x][p.y].animal = NEWANIMAL(ROCK, -1, -1);
	}
	else
		matrix[p.x][p.y].animal = NEWANIMAL(EMPTY, -1, -1);
	return 1;
}

Overwriting types.c


In [ ]:
%%writefile config.ini

2 4 3 6 5 5 9
ROCA 0 0
CONEJO 0 2
ZORRO 0 4
ZORRO 1 0
ZORRO 1 4
ROCA 2 4
CONEJO 3 0
CONEJO 4 0
ZORRO 4 4

Overwriting config.ini


In [ ]:
%%writefile main.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <omp.h>
#include "headers/fileHandler.h"
#include "headers/config_t.h"
#include "headers/types_t.h"
#include "headers/rules.h"
#include "headers/coord_t.h"
#include "fileHandler.c"
#include "rules.c"
#include "types.c"

void printResult (object_t **eco, config_t conf);
void destroyLock (object_t **eco, config_t conf);

int main (int argc, char **argv) {
	if (argc < 2) {
		printf("Usage:\n%s <config_file>\n\n", argv[0]);
		exit(1);
	}

	config_t conf;

	object_t **eco;
	coord_t p;

	printf("Leyendo archivo");
	readFile(argv[1], &conf, &eco);

	#ifdef STEP
		system("clear");
		printf("-------GEN:%2d-------\n", 0);
		printMatrix (eco, conf.L, conf.C, conf);
	#endif

	for (unsigned int t = 1; t <= conf.N_GEN; t++){
		conf.GEN++;

		#ifdef STEP
		getchar();
		system("clear");
		printf("-------GEN:%2d-------\n", t);
		#endif

		// REGRAS DO CONEJO
		for (unsigned int i = 0; i < conf.L; i++){
			for (unsigned int j = 0; j < conf.C; j++){
				if (eco[i][j].animal.type == RABBIT){
					p = (coord_t){.x = i, .y = j};
					rabbit_rules (eco, conf, p);
				}	
			}
		}

		//CONFLITOS DO CONEJO
		#ifdef PARALELO
		#pragma omp parallel for   //paralelo
		#endif
		for (unsigned int i = 0; i < conf.L; i++){
			for (unsigned int j = 0; j < conf.C; j++){
				eco[i][j].animal = choose_rabbit(eco[i][j]);
			}
		}

		#ifdef STEP2
			printMatrix (eco, conf.L, conf.C, conf);
			getchar();
			system("clear");
			printf("-------GEN:%2d-------\n", t);
		#endif

		//REGRAS DA ZORRO
		for (unsigned int i = 0; i < conf.L; i++){
			for (unsigned int j = 0; j < conf.C; j++){
				if (eco[i][j].animal.type == FOX){
					p = (coord_t){.x = i, .y = j};
					fox_rules (eco, conf, p);
				}	
			}
		}

		//CONFLITOS DA ZORRO
		#ifdef PARALELO
		#pragma omp parallel for    //paralelo
		#endif
		for (unsigned int i = 0; i < conf.L; i++){
			for (unsigned int j = 0; j < conf.C; j++){
				eco[i][j].animal = choose_fox(eco[i][j]);
			}
		}

		printf("\rGeneración %d/%d", t, conf.N_GEN);
		fflush(stdout);
		#ifdef STEP
			printMatrix (eco, conf.L, conf.C, conf);
		#endif
	}

	//printResult(eco, conf);
	return 0;
}


void printResult (object_t **eco, config_t conf) {
	int N = 0;
	for (unsigned int i = 0; i < conf.L; i++)
		for (unsigned int j = 0; j < conf.C; j++)
			if (eco[i][j].animal.type != EMPTY){
				N++;
			}

	printf("%u %u %u 0 %u %u %u\n",conf.GEN_PROC_CONEJOS,
									conf.GEN_PROC_ZORROS,
									conf.GEN_COMIDA_ZORROS,
									conf.L,
									conf.C,
									N);
	for (unsigned int i = 0; i < conf.L; i++)
		for (unsigned int j = 0; j < conf.C; j++)
			if (eco[i][j].animal.type != EMPTY){
				if (eco[i][j].animal.type == FOX) {
					printf("ZORRO %d %d\n", i, j);
				}
				else if (eco[i][j].animal.type == RABBIT) {
					printf("CONEJO %d %d\n", i, j);
				}
				else if (eco[i][j].animal.type == ROCK) {
					printf("ROCA %d %d\n", i, j);
				}
			}
}

Overwriting main.c


In [ ]:
!gcc -Wall -g -o prey-predator -fopenmp main.c
!./prey-predator config.ini

Generación 6/6